In [2]:
import pandas as pd
import numpy as np
from ofxparse import OfxParser
import matplotlib.pyplot as plt
import os
import tabula

In [14]:
path = "statements/Statement_jan11_mar31.pdf"
# statements/Statement_apr01_jun30.pdf
# statements/Statement_jul01_sep30.pdf  

df1 = tabula.read_pdf(path, area=(426, 57, 784, 541), pages=1, silent=True)[0]
df2 = tabula.read_pdf(path, area=(207, 57, 774, 541), pages=2, silent=True)[0]
df3 = tabula.read_pdf(path, area=(102, 57, 708, 539), pages=3, silent=True)[0]

print(df1.columns)
print(df2.columns)
print(df3.columns)


Index(['Date', 'Transaction', 'Debit', 'Unnamed: 0', 'Credit', 'Balance'], dtype='object')
Index(['Date Transaction', 'Unnamed: 0', 'Debit', 'Unnamed: 1', 'Credit',
       'Balance'],
      dtype='object')
Index(['Date Transaction', 'Unnamed: 0', 'Debit', 'Unnamed: 1', 'Credit',
       'Balanc'],
      dtype='object')


In [25]:
df1 = df1.fillna(0)
df2 = df2.fillna(0)
df3 = df3.fillna(0)

try:
    df1 = df1.drop(df1.filter(regex='Unnamed').columns, axis=1)
    df2 = df2.drop(df2.filter(regex='Unnamed').columns, axis=1)
    df3 = df3.drop(df3.filter(regex='Unnamed').columns, axis=1)
except:
    pass

try:
    df1 = df1.drop('Balance', axis=1)
    df2 = df2.drop('Balance', axis=1)
    df3 = df3.drop('Balance', axis=1)
except:
    pass

print(df1.columns)
print(df2.columns)
print(df3.columns)


Index(['Date', 'Transaction', 'Debit', 'Credit'], dtype='object')
Index(['Date Transaction', 'Debit', 'Credit'], dtype='object')
Index(['Date Transaction', 'Debit', 'Credit', 'Balanc'], dtype='object')


In [19]:
df3

,Date Transaction,Debit,Credit,Balanc
0,07 Mar POS 15533600 07 MAR,0,$118.50,$994.13 C
1,08 Mar POS 15533600 08 MAR,0,$30.00,"$1,024.13 C"
2,09 Mar POS 15533600 09 MAR,0,$213.00,"$1,237.13 C"
3,10 Mar POS 15533600 10 MAR,0,$223.00,"$1,460.13 C"
4,11 Mar POS 15533600 11 MAR,0,$129.00,"$1,589.13 C"
5,12 Mar POS 15533600 12 MAR,0,$12.50,"$1,601.63 C"
6,13 Mar POS 15533600 13 MAR,0,$267.00,"$1,868.63 C"
7,14 Mar POS 15533600 14 MAR,0,$115.00,"$1,983.63 C"
8,15 Mar POS 15533600 15 MAR,0,$331.00,"$2,314.63 C"
9,16 Mar POS 15533600 16 MAR,0,$101.00,"$2,415.63 C"


In [35]:
# get downloaded statments
def get_files(filetype):
    files = []
    for f in os.listdir('statements/'):
        if not f.startswith('.') and ("." + filetype) in f: 
            files.append("statements/" + f)
    
    return files

In [ ]:
# get tables from PDF
def get_tables(file, pages="all", multiple_tables=True, debug=True, area=()):
    tables = tabula.read_pdf(file, 
                             multiple_tables=multiple_tables, 
                             pages=pages, 
                             silent=debug  # suppres stdoutput
                             )

    # drop unnamed columns
    for i in range(len(tables)):
        tables[i].fillna("", inplace=True)

        if 'Balance' in tables[i].iloc[0].tolist():
            # swap first row for headers, then drop
            tables[i].columns = [str(i) for i in tables[i].iloc[0]]
            tables[i] = tables[i].iloc[1:]

        # drop empty columns
        tables[i].drop(tables[i].filter(regex='Unnamed|nan').columns, axis=1, inplace=True)



    # join into one table
    frames = [f for f in tables if len(f.columns) == 5]
    table = pd.concat(frames)
    table.columns = [col.lower() for cols in table.columns]

    # clean up the rows
    # table.drop()
    
    
    return table

In [3]:
# returnt he transaction from .ofx file type
def get_transactions(filename):
    with open(filename) as f:
        ofx = OfxParser.parse(f)

    account = ofx.account
    statement = account.statement
    transactions = statement.transactions

    return transactions

In [4]:
# build dataframe
df = pd.DataFrame(columns=['date', 'year', 'month', 'day', 'description', 'debit', 'credit'])

files = get_files("ofx")

for f in files:
    for t in get_transactions(f):
        debit = t.amount if t.type == "debit" else 0
        credit = t.amount if t.type == "credit" else 0
        new = {'date':t.date, 'year': t.date.year, 'month': t.date.month, 'day': t.date.day, 'description': t.memo, 'debit': debit, 'credit': credit}
        
        df = df.append(new, ignore_index=True)

df = df.astype({"description": str, "debit": float, "credit": float})

In [ ]:
# save the data frame for analysis
df.to_pickle("./transaction_2018_current.pkl")

